In [46]:
from whoosh.index import create_in, open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser, FuzzyTermPlugin
import os


import requests

In [35]:
def create_search_index(directory):
    schema = Schema(title=ID(stored=True), content=TEXT)
    if not os.path.exists("indexdir"):
        os.mkdir("indexdir")
    ix = create_in("indexdir", schema)
    writer = ix.writer()
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                writer.add_document(title=filename, content=content)
    
    writer.commit()

def search_in_index(search_pattern):
    result_files = []
    ix = open_dir("indexdir")
    with ix.searcher() as searcher:
        query = QueryParser("content", ix.schema).parse(search_pattern)
        results = searcher.search(query)
        for result in results:
            result_files.append(result['title'])

    return result_files


def search_in_index_2(search_pattern):
    result_files = []
    ix = open_dir("indexdir")
    with ix.searcher() as searcher:
        parser = QueryParser("content", ix.schema)
        parser.add_plugin(FuzzyTermPlugin())
        query = parser.parse(f"{search_pattern}~2")  # "~2" makes it a fuzzy query with a max edit distance of 2
        results = searcher.search(query)
        for result in results:
            result_files.append(result['title'])
    return result_files

In [36]:
directory_path = '../data/rag/files'
create_search_index(directory_path)

In [44]:
search_query = 'я хочу изменить сроки выполнения через доп соглашение'
search_pattern = 'сроки'
search_results = search_in_index(search_pattern)

In [ ]:
search_results

In [ ]:
def read_file(file_path):
    with open(file_path, 'r') as f:
        return f.read()
    
context = read_file(os.path.join(directory_path, search_results[0]))